In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold,train_test_split
import gc


In [2]:
label_address = "./data/label.csv"
test_address = "./data/test_set.csv"
train_address = "./data/train_set.csv"
orders_address = "./data/orders.csv"

In [3]:
train = pd.read_csv(train_address, index_col=0)
label = pd.read_csv(label_address, index_col=0)
test = pd.read_csv(test_address, index_col=0)
orders = pd.read_csv(orders_address)

/Users/yuki/.local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:

orders.head()
test2 = orders[orders.eval_set =='test']
label["label"] = label["label"].astype(np.float32)
label = label["label"].values.tolist()


In [5]:
x_train = train
y_train = test
x_test = label

In [6]:
x_train.shape


(8474661, 20)

In [7]:
x_train = np.array(x_train)
x_train
x_train.shape

(8474661, 20)

In [8]:
y_train = np.array(y_train)
y_train
y_train.shape

(4833292, 20)

In [9]:
x_test = np.array(x_test)
x_test

array([0., 1., 1., ..., 0., 0., 0.])

In [10]:
x_test.shape

(8474661,)

In [11]:
#x_train = train
#y_train = test
#x_test = label

In [12]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
        params = {}
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'logloss'
        params["eta"] = 0.05
        params["subsample"] = 0.7
        params["min_child_weight"] = 10
        params["colsample_bytree"] = 0.7
        params["max_depth"] = 8
        params["silent"] = 1
        params["seed"] = seed_val
        num_rounds = 100
        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=10)
        else:
                xgtest = xgb.DMatrix(test_X)
                model = xgb.train(plst, xgtrain, num_rounds)

        pred_test_y = model.predict(xgtest)
        return pred_test_y

In [13]:
pred = runXGB(x_train, x_test, y_train)

[23:31:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:31:20] WARNING: /Users/travis/build/dmlc/xgboost/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [14]:
#cutoff = 0.2
#pred[pred>=cutoff] = 1
#pred[pred<cutoff] = 0

In [15]:
test["result"] = pred

In [16]:
#test

In [17]:
TRESHOLD = 0.22 
d = dict()
for row in test.itertuples():
    if row.result > TRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)
for order in test2.order_id:
    if order not in d:
        d[order] = 'None'
        
submission = pd.DataFrame.from_dict(d, orient='index')

submission.reset_index(inplace=True)
submission.columns = ['order_id', 'products']
submission.to_csv('submission_improved_xgboost.csv', index=False)

In [18]:
xgbo = pd.read_csv("./submission_improved_xgboost.csv", index_col=0)
xgbo

,products
order_id,
2774568,21903 21137 47766 9387
1528013,21903
1376945,13176
2161313,196
1416320,24852
1980631,13575 6184 9387 46061 13914 41400 22362
3202221,24852
3222866,40706 13187 35458 37507 37131 32912 49680 7969...
707453,45066 21137 47766 24852 21903
